<a href="https://colab.research.google.com/github/hariprasadpl088/Resume-Checker-Using-Langgraph/blob/main/Resume_Checker_using_Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph
!pip install langchain_google_genai

In [ ]:
from typing import TypedDict,Optional
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [ ]:
class GraphState(TypedDict):
  resume: Optional[str]
  qualified: Optional[bool]
  message: Optional[str]

In [ ]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.7)

In [ ]:
def load_resume(state):
  resume= """Name: Hari Prasad PL
  skills: Python,PowerBI,Machine learning,Deep Learning
  Experience: 1 year
  Degree: B.Tech Artificial Intelligence and Data Science"""
  return {"resume": resume}

In [ ]:
def check_qualification(state):
  resume=state["resume"]
  prompt=f"""Based on this resume, is the candidate eligible for the AI/ML role?Answer only YES or NO.\n\n{resume}"""
  response=llm.invoke ([HumanMessage(content=prompt)])
  qualified = "YES" in response.content.upper()
  return {"resume": resume,"qualified": qualified}

In [ ]:
def write_positive(state):
  resume=state["resume"]
  prompt=f"Write a positive reply for this qualified candidate:\n{resume}"
  response=llm.invoke([HumanMessage(content=prompt)])
  return {"message": response.content}


In [ ]:
def write_rejection(state):
  resume=state["resume"]
  prompt=f"Politely reject this candidate:\n{resume}"
  response=llm.invoke([HumanMessage(content=prompt)])
  return {"message": response.content}

In [ ]:
def route_decision(state):
  return "positive_feedback" if state["qualified"] else "rejection_message"

In [ ]:
builder=StateGraph(GraphState)
builder.add_node("load_resume",load_resume)
builder.add_node("check_qualification",check_qualification)
builder.add_node("positive_feedback",write_positive)
builder.add_node("rejection_message",write_rejection)
builder.set_entry_point("load_resume")
builder.add_edge("load_resume","check_qualification")
builder.add_conditional_edges("check_qualification",route_decision)
builder.set_finish_point("positive_feedback")
builder.set_finish_point("rejection_message")

In [ ]:
graph=builder.compile()
output=graph.invoke({})

In [ ]:
print("resume:")
print(output["resume"])
print("\n result:","Qualified" if output["qualified"] else "Not Qualified")
print("\n Final message")
print(output["message"])

resume:
Name: Hari Prasad PL
  skills: Python,PowerBI,Machine learning,Deep Learning
  Experience: 1 year
  Degree: B.Tech Artificial Intelligence and Data Science

 result: Not Qualified

 Final message
Subject: Update on your Application for [Job Title] at [Company Name]

Dear Hari Prasad PL,

Thank you for your interest in the [Job Title] position at [Company Name] and taking the time to submit your application. We appreciate you sharing your skills and experience with us.

We have reviewed your application and qualifications, including your experience with Python, Power BI, machine learning, and deep learning.  While your background is promising, we have decided to move forward with other candidates whose experience and qualifications more closely align with the specific requirements of this role.

We wish you the best in your job search and future endeavors.

Sincerely,

The [Company Name] Hiring Team
